In [1]:
#importing packages
import pandas as pd
from bs4 import BeautifulSoup
import requests
import string
import sys
import concurrent.futures
import time


In [2]:
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"}

### Downloading body text

In [3]:
t1 = time.perf_counter()
final = {}

def download_text(url):
    try:
        global info
        info = {}
        r = requests.get(url, headers = headers)
        time.sleep(10)
        html_doc= r.text
        soup = BeautifulSoup(html_doc, 'html.parser')
        #print(soup.prettify())
        if soup.find_all('p'):
            y =""
            for x in soup.find_all('p'):
                y += x.get_text()
            info[soup.title.text] = y
        elif soup.find_all('section'):
            y = ""
            for x in soup.find_all('section'): #section class = 'articlebody'
                y += x.get_text()
            info[soup.title.text] = y
    except:
        e = sys.exc_info()[0]
        print(e)
        print(url)
    
    final.update(info)
  
  
for chunk in pd.read_csv("us_articles_final.csv",chunksize = 50):   #INPUT CSV FILE HERE 
    with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(download_text, chunk.iloc[:,3])
        
    
t2 = time.perf_counter()
print(f'Finished in {t2-t1} seconds')

#Final ouput is dictionary called 'final'

<class 'requests.exceptions.ConnectionError'>
https://amp.theguardian.comsociety/2020/jun/16/private-hospitals-can-help-nhs-recover-from-covid-19
<class 'requests.exceptions.ConnectionError'>
https://amp.theguardian.comsociety/2020/jun/27/jobless-total-to-hit-1980s-levels-without-fresh-state-support
<class 'requests.exceptions.ConnectionError'>
https://amp.theguardian.comus-news/2020/jul/09/us-coronavirus-cases-record-fauci
<class 'requests.exceptions.ConnectionError'>
https://amp.theguardian.comus-news/2020/jun/17/gender-roles-parenting-housework-coronavirus-pandemic
<class 'requests.exceptions.ConnectionError'>
https://amp.theguardian.comus-news/2020/jun/29/cameron-webb-obama-history-congress-virignia
<class 'requests.exceptions.ConnectionError'>
https://amp.theguardian.comus-news/2020/jul/03/los-angeles-covid-19-surge-homeless-immigrants
<class 'requests.exceptions.ConnectionError'>
https://amp.theguardian.comworld/2020/jun/29/abortion-ruling-supreme-court-strikes-down-louisiana-law

In [4]:
#confirmation that we have all/most the articles
i = pd.read_csv('us_articles_final.csv')
print('The length of the original dataset is ' , len(i))
print("\n")
print('The numer of articles we succesfully grabbed body text from is ', len(final))

The length of the original dataset is  1952


The numer of articles we succesfully grabbed body text from is  1843


In [6]:
#dictionary to dataframe
df = pd.DataFrame(list(final.items()),columns = ['title','body_text']) 

In [7]:
#dataframe to csv
df.to_csv("Articles_text.csv",index = False)